### Importing essential libraries 

In [35]:
import pandas as pd 
import numpy as np 
import requests 
from bs4 import BeautifulSoup
from tqdm import tqdm

link = 'https://github.com/AshishJangra27'

soup = BeautifulSoup(requests.get(link).content,'html.parser')

### Defining a social account functions 

In [2]:
def social_account(urls):
    accounts = []
    for url in urls:
        if 'kaggle' in url:
            accounts.append('Kaggle')
        elif 'instagram' in url:
            accounts.append('Instagram')
        elif 'facebook' in url:
            accounts.append('Facebook')
        elif 'linkedin' in url:
            accounts.append('Linkedin')
        elif 'github' in url:
            accounts.append('Github')
        else:
            accounts.append('Personal website')
    return ','.join(accounts)

### Scraping the data of the github repositories 

In [3]:
user_name  = soup.find('span',class_ = 'p-name vcard-fullname d-block overflow-hidden').text.strip()
user_id    = soup.find('span',class_ = 'p-nickname vcard-username d-block').text.strip()
bio        = soup.find('div',class_ = 'js-profile-editable-area d-flex flex-column d-md-block').find('div').text
lives_in   = soup.find('li',class_ = 'vcard-detail pt-1 hide-sm hide-md').text.strip()
data = []

for i in soup.find_all('a',class_ = 'Link--secondary no-underline no-wrap'):
    if 'follower' in (i.text):
        follower = int(i.find('span').text.strip())
    elif 'following' in i.text:
        following = int(i.find('span').text.strip())
    try:        
        content  = [int(content.find('span').text.strip()) for content in soup.find('nav',class_ ='UnderlineNav-body width-full p-responsive js-sidenav-container-pjax').find_all('a')[1:]]
        repos    = content[0]
        project  = content[1]
        packages = content[2]
        stars    = content[3]
    except:
        repos    = np.nan
        project  = np.nan
        packages = np.nan
        stars    = np.nan
    try:
        url = [links.find('a')['href'] for links in soup.find_all('li',class_='vcard-detail pt-1')]
        account = social_account(url)
    except:
        url = np.nan
        account = 'no social account has been linked'


data.append([user_id , user_name , bio , follower , following ,lives_in,url,account, repos , project , packages , stars])

df = pd.DataFrame(data,columns = ['user_id' ,'user_name' ,'bio' ,'follower' ,'following', 'lives_in', 'url','account', 'repos' , 'project' , 'packages' , 'stars'])
df.head()




,user_id,user_name,bio,follower,following,lives_in,url,account,repos,project,packages,stars
0,AshishJangra27,Ashish Jangra,I am an AI Trainer. \nI train students on tech...,303,1,Noida,[],,76,0,0,52


### Scraping the repos

In [4]:
link = 'https://github.com/AshishJangra27?tab=repositories'
repos_links = []
repos_name  = []
for pages in range(int(round(df['repos']/30))):
    res = requests.get(link)
    soup = BeautifulSoup(res.content,'html.parser')
    for name in soup.find_all('div',class_ ='d-inline-block mb-1'):
        repos_links.append('https://www.github.com' + name.find('a').get('href'))
    for name in soup.find_all('div',class_ ='d-inline-block mb-1'):
        repos_name.append(name.find('a').text.strip())
    try:
        link = 'https://www.github.com' + soup.find('div',{'class':'paginate-container'}).find('a',class_ = 'next_page').get('href')    
    except:
        pass

### Scraping the stars

In [5]:
link = 'https://github.com/AshishJangra27?tab=stars'

stars_repo_link = []
stars_repo_name = []

for pages in range(int(round(df['stars']/30))):
    res = requests.get(link).text
    soup = BeautifulSoup(res,'html.parser')
    for stars in soup.find_all('div',class_ = 'd-inline-block mb-1'):
        stars_repo_link.append('https://www.github.com' + stars.find('a').get('href'))
    for stars in soup.find_all('div',class_ = 'd-inline-block mb-1'):
        stars_repo_name.append(stars.find('a').text.strip().split('/')[1].strip())
    try:
        link = soup.find('div',{'class':'paginate-container'}).find('a',class_ = 'next_page').get('href')
    except:
        pass

### Concating the data 


In [6]:
df_ = pd.DataFrame()

df_['repos_name']       = [repos_name]
df_['repos_links']      = [repos_links]
df_['stars_repos']      = [stars_repo_link]
df_['stars_repos_link'] = [stars_repo_name]

df = pd.concat([df,df_],axis = 1)
df

,user_id,user_name,bio,follower,following,lives_in,url,account,repos,project,packages,stars,repos_name,repos_links,stars_repos,stars_repos_link
0,AshishJangra27,Ashish Jangra,I am an AI Trainer. \nI train students on tech...,303,1,Noida,[],,76,0,0,52,"[In-One-Go, Data-Science-Live-GFG, Airbnb-Scra...",[https://www.github.com/AshishJangra27/In-One-...,[https://www.github.com/AshishJangra27/Gaana-D...,"[Gaana-Data-Analysis, GFG-Tableau-Course-Datas..."


### Scraping the followers

In [20]:
user_id_follower  = []
follower_username = []
for pages in range(1,int(round(df['follower']/50))+1):
    link = 'https://github.com/AshishJangra27?page=' + str(pages) + '&tab=followers'
    res = requests.get(link).text
    soup = BeautifulSoup(res,'html.parser')
    for followers in soup.find_all('div',class_ = 'd-table table-fixed col-12 width-full py-4 border-bottom color-border-muted'):
        user_id_follower.append(followers.find_all('span')[1].text)
    for name in soup.find_all('div',class_ = 'd-table table-fixed col-12 width-full py-4 border-bottom color-border-muted'):
        follower_username.append(name.find_all('span')[0].text.strip())



In [26]:
followers = pd.DataFrame({'user_id':user_id_follower,'username':follower_username})
followers

,user_id,username
0,imayank45,Mayank Kathane
1,ChidambaraRaju,Chidambara Raju
2,prashant-3200,
3,Nandhinikumar223,NANDHINI K
4,Shaikabdulsalman,
...,...,...
295,charishmaseera,Charishma Seera
296,shameerjs93,
297,solankishashank,
298,pranay-creator,Pranay sharma


In [ ]:
# data of the followers 

In [30]:
for i in followers['user_id']:
    print('https://www.github.com/'+i)

https://www.github.com/imayank45
https://www.github.com/ChidambaraRaju
https://www.github.com/prashant-3200
https://www.github.com/Nandhinikumar223
https://www.github.com/Shaikabdulsalman
https://www.github.com/VishalEmmanuel
https://www.github.com/snehil943
https://www.github.com/harshitpathak027
https://www.github.com/AryadeepIT
https://www.github.com/abhisekdas1201
https://www.github.com/Abhishek4u247
https://www.github.com/Priyanka-Kanumuri
https://www.github.com/prathm1008
https://www.github.com/AdityaPratap-129
https://www.github.com/imakshita9
https://www.github.com/Aniket760
https://www.github.com/AnanthuVijay
https://www.github.com/invincible267
https://www.github.com/elahi9294
https://www.github.com/Abd8603
https://www.github.com/Yogeshkumar170201
https://www.github.com/200390107061
https://www.github.com/12Gour
https://www.github.com/parteon
https://www.github.com/rohit-kumar170
https://www.github.com/Abhikhush28
https://www.github.com/Sukomal07
https://www.github.com/tahirm

In [41]:
data = []

for i in tqdm(followers['user_id']):
    link = 'https://www.github.com/'+i
    soup = BeautifulSoup(requests.get(link).content,'html.parser')
    user_name  = soup.find('span',class_ = 'p-name vcard-fullname d-block overflow-hidden').text.strip()
    user_id    = soup.find('span',class_ = 'p-nickname vcard-username d-block').text.strip()
    try:
        bio        = soup.find('div',class_ = 'js-profile-editable-area d-flex flex-column d-md-block').find('div').text
    except:
        bio        = 'No bio written'
    try:
        lives_in   = soup.find('li',class_ = 'vcard-detail pt-1 hide-sm hide-md').text.strip()
    except:
        lives_in   = np.nan

    for i in soup.find_all('a',class_ = 'Link--secondary no-underline no-wrap'):
        if 'follower' in (i.text):
            follower = i.find('span').text.strip()
        elif 'following' in i.text:
            following = i.find('span').text.strip()
        try:        
            content  = [int(content.find('span').text.strip()) for content in soup.find('nav',class_ ='UnderlineNav-body width-full p-responsive js-sidenav-container-pjax').find_all('a')[1:]]
            repos    = content[0]
            project  = content[1]
            packages = content[2]
            stars    = content[3]
        except:
            repos    = np.nan
            project  = np.nan
            packages = np.nan
            stars    = np.nan
        try:
            url = [links.find('a')['href'] for links in soup.find_all('li',class_='vcard-detail pt-1')]
            account = social_account(url)
        except:
            url = np.nan
            account = 'no social account has been linked'


    data.append([user_id , user_name , bio , follower , following ,lives_in,url,account, repos , project , packages , stars])

followers_data = pd.DataFrame(data,columns = ['user_id' ,'user_name' ,'bio' ,'follower' ,'following', 'lives_in', 'url','account', 'repos' , 'project' , 'packages' , 'stars'])
followers_data.head()

100%|█████████████████████████████████████████| 300/300 [07:02<00:00,  1.41s/it]


,user_id,user_name,bio,follower,following,lives_in,url,account,repos,project,packages,stars
0,imayank45,Mayank Kathane,,0,1,NaN,[],,22.0,0.0,0.0,3.0
1,ChidambaraRaju,Chidambara Raju,A Data Alchemist who Transforms Raw Data into ...,4,5,NaN,[https://www.kaggle.com/rajugc],Kaggle,14.0,0.0,0.0,34.0
2,prashant-3200,,,0,1,NaN,[],,0.0,0.0,0.0,0.0
3,Nandhinikumar223,NANDHINI K,I am a Mechanical Engineering Student in Rajal...,3,5,Rajalakshmi Engineering College,NaN,no social account has been linked,7.0,0.0,0.0,0.0
4,Shaikabdulsalman,,,1,1,NaN,NaN,no social account has been linked,0.0,0.0,0.0,4.0


In [47]:
followers_data[followers_data['user_id']=='RahulWadwani']

,user_id,user_name,bio,follower,following,lives_in,url,account,repos,project,packages,stars
71,RahulWadwani,Rahul,I am Rahul Wadwani. A Student at Mumbai Univer...,1,5,New panvel,[https://www.kaggle.com/rahulwadwani],Kaggle,7.0,0.0,0.0,9.0
